In [1]:
% env CUDA_VISIBLE_DEVICES=2

env: CUDA_VISIBLE_DEVICES=2


In [2]:
import tensorflow as tf
import tensorflow_hub as hub

In [5]:
questions = open('../data/quora/quora.txt').read().splitlines()

In [6]:
embedder_url = "https://tfhub.dev/google/universal-sentence-encoder/2"

with tf.Graph().as_default():
    embed = hub.Module(embedder_url)
    embeddings = embed(questions)

    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        sess.run(tf.tables_initializer())

        embeddings_vec = sess.run(embeddings)

INFO:tensorflow:Using /tmp/tfhub_modules to cache modules.
INFO:tensorflow:Downloading TF-Hub Module 'https://tfhub.dev/google/universal-sentence-encoder/2'.
INFO:tensorflow:Downloaded TF-Hub Module 'https://tfhub.dev/google/universal-sentence-encoder/2'.
INFO:tensorflow:Initialize variable module/Embeddings_en/sharded_0:0 from checkpoint b'/tmp/tfhub_modules/1fb57c3ffe1a38479233ee9853ddd7a8ac8a8c47/variables/variables' with Embeddings_en/sharded_0
INFO:tensorflow:Initialize variable module/Embeddings_en/sharded_1:0 from checkpoint b'/tmp/tfhub_modules/1fb57c3ffe1a38479233ee9853ddd7a8ac8a8c47/variables/variables' with Embeddings_en/sharded_1
INFO:tensorflow:Initialize variable module/Embeddings_en/sharded_10:0 from checkpoint b'/tmp/tfhub_modules/1fb57c3ffe1a38479233ee9853ddd7a8ac8a8c47/variables/variables' with Embeddings_en/sharded_10
INFO:tensorflow:Initialize variable module/Embeddings_en/sharded_11:0 from checkpoint b'/tmp/tfhub_modules/1fb57c3ffe1a38479233ee9853ddd7a8ac8a8c47/var

In [10]:
import numpy as np

np.save('../data/embeddings_from_mt', embeddings_vec)

In [11]:
# Let's free GPU before using our VAE pytorch model
sess.close()

In [17]:
from torchtext import data
from torchtext.data import Field, Dataset, Example

batch_size = 64

questions_bpe = open('../data/quora/quora.tok.bpe').read().splitlines()
text = Field(init_token='<bos>', eos_token='<eos>', batch_first=True)
examples = [Example.fromlist([s], [('text', text)]) for s in questions_bpe]
dataset = Dataset(examples, [('text', text)])
text.build_vocab(dataset)
dataloader = data.BucketIterator(dataset, batch_size, train=False, sort=False)

In [18]:
import sys; sys.path.append('..')

import torch
import numpy as np
from tqdm import tqdm
from firelab.utils import cudable

from src.models import VAE, LSTMEncoder, LSTMDecoder

embeddings = []
emb_size = 1024
hid_size = 1024
latent_size = 64

encoder = LSTMEncoder(emb_size, hid_size, len(text.vocab), latent_size)
decoder = LSTMDecoder(emb_size, hid_size, len(text.vocab), latent_size)
vae = cudable(VAE(encoder, decoder, latent_size))
vae.load_state_dict(torch.load('../experiments/zoo/checkpoints/vae-134352.pth'))

def embeddings_from_batch(batch):
    batch.text = cudable(batch.text)
    inputs, trg = batch.text[:, :-1], batch.text[:, 1:]
    encodings = vae.encoder(inputs)
    means = encodings[:, :vae.latent_size]
    
    return means
    
for batch in dataloader:
    embeddings.append(embeddings_from_batch(batch).detach().cpu().numpy())

/home/universome/pyvenvs/zoo/lib/python3.6/site-packages/torchtext/data/field.py:322: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  return Variable(arr, volatile=not train)


In [20]:
np.save('../data/embeddings_from_vae', np.concatenate(embeddings))